In [1]:
import random
import gc
from math import floor, ceil

from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from scipy.stats import spearmanr

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoConfig, AutoTokenizer, AutoModel, AdamW

pd.set_option('max_column', 50)
gc.collect()

22

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_df = pd.read_csv('../input/google-quest-challenge/train.csv')

print(train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6079 entries, 0 to 6078
Data columns (total 41 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   qa_id                                  6079 non-null   int64  
 1   question_title                         6079 non-null   object 
 2   question_body                          6079 non-null   object 
 3   question_user_name                     6079 non-null   object 
 4   question_user_page                     6079 non-null   object 
 5   answer                                 6079 non-null   object 
 6   answer_user_name                       6079 non-null   object 
 7   answer_user_page                       6079 non-null   object 
 8   url                                    6079 non-null   object 
 9   category                               6079 non-null   object 
 10  host                                   6079 non-null   object 
 11  ques

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [4]:
test_df = pd.read_csv('../input/google-quest-challenge/test.csv')

print(test_df.info())
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   qa_id               476 non-null    int64 
 1   question_title      476 non-null    object
 2   question_body       476 non-null    object
 3   question_user_name  476 non-null    object
 4   question_user_page  476 non-null    object
 5   answer              476 non-null    object
 6   answer_user_name    476 non-null    object
 7   answer_user_page    476 non-null    object
 8   url                 476 non-null    object
 9   category            476 non-null    object
 10  host                476 non-null    object
dtypes: int64(1), object(10)
memory usage: 41.0+ KB
None


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [5]:
sub = pd.read_csv('../input/google-quest-challenge/sample_submission.csv')
TARGET_COLUMNS = sub.columns.values[1:].tolist()
TARGET_COLUMNS
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [6]:
MAX_LEN = 512
SEP_TOKEN = 102

class QUESTDataset(Dataset):
    def __init__(self, df, tokenizer, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input_ids, seg_ids, attention_mask = self.get_token_ids(row)
#         attention_mask = (input_ids > 0)
        if self.labeled:
            labels = self.get_label(row)
            return input_ids, seg_ids, attention_mask, labels
        else:
            return input_ids, seg_ids, attention_mask

    def __len__(self):
        return len(self.df)
    
    def select_tokens(self, tokens, max_num):
        if len(tokens) <= max_num:
            return tokens
        if self.train_mode:
            num_remove = len(tokens) - max_num
            remove_start = random.randint(0, len(tokens)-num_remove-1)
            return tokens[:remove_start] + tokens[remove_start + num_remove:]
        else:
            return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]
            
    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=238, a_max_len=238):
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)
        t_len = len(t)
        q_len = len(q)
        a_len = len(a)
        
        if (t_len+q_len+a_len+6) > max_sequence_length:
            if t_max_len > t_len:
                t_new_len = t_len
                a_max_len = a_max_len + floor((t_max_len - t_len)/2)
                q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
            else:
                t_new_len = t_max_len

            if a_max_len > a_len:
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+6 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+6)))

            if t_len > t_new_len:
                ind1 = floor(t_new_len/2)
                ind2 = ceil(t_new_len/2)
                t = t[:ind1]+t[-ind2:]
            else:
                t = t[:t_new_len]

            if q_len > q_new_len:
                ind1 = floor(q_new_len/2)
                ind2 = ceil(q_new_len/2)
                q = q[:ind1]+q[-ind2:]
            else:
                q = q[:q_new_len]

            if a_len > a_new_len:
                ind1 = floor(a_new_len/2)
                ind2 = ceil(a_new_len/2)
                a = a[:ind1]+a[-ind2:]
            else:
                a = a[:a_new_len]

        return t, q, a

    def get_attention_masks(self, tokens):
        if len(tokens) > 512:
            raise IndexError("Token length more than max seq length!")
        return [1]*len(tokens) + [0] * (512 - len(tokens))

    def get_seg_ids(self, ids):
        seq_ids = torch.zeros_like(ids)
        seq_idx = 0
        first_sep = True
        sep_token = self.tokenizer.sep_token
        for i, e in enumerate(ids):
            seq_ids[i] = seq_idx
            if e == sep_token:
                if first_sep:
                    first_sep = False
                else:
                    seq_idx = 1
        pad_idx = torch.nonzero(ids == 0)
        seq_ids[pad_idx] = 0

        return seq_ids
    
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)
        tokens = ['<s>'] + t_tokens + ['</s>','</s>'] + q_tokens + ['</s>','</s>'] + a_tokens + ['</s>']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            token_ids += [self.tokenizer.pad_token_id] * (MAX_LEN - len(token_ids)) # padding
        
        ids = torch.tensor(token_ids)
        seg = self.get_seg_ids(ids)
        attention_masks = self.get_attention_masks(tokens)
        attention_masks = torch.tensor(attention_masks)
        return ids, seg, attention_masks

    def get_label(self, row):
        return torch.tensor(row[TARGET_COLUMNS].values.astype(np.float32))
    
    def collate_fn(self, batch):
        input_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
        attention_mask = torch.stack([x[2] for x in batch])
        
        if self.labeled:
            labels = torch.stack([x[3] for x in batch])
            return input_ids, seg_ids, attention_mask, labels
        else:
            return input_ids, seg_ids, attention_mask

In [7]:
def get_test_loader(tokenizer, batch_size=4):
    test_dataset = QUESTDataset(test_df, tokenizer, train_mode=False, labeled=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=test_dataset.collate_fn, drop_last=False)
    test_loader.num = len(test_df)
    
    return test_loader


def get_train_val_loaders(df, tokenizer, batch_size=4, val_batch_size=4, ifold=0):
    dataset = QUESTDataset(df, tokenizer)
    custom_collat_fn = dataset.collate_fn
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=custom_collat_fn, drop_last=True)
    loader.num = len(loader)

    return loader

In [8]:
class QUESTModel(nn.Module):
    def __init__(self, transformer):
        super(QUESTModel, self).__init__()
        self.roberta = AutoModel.from_pretrained(transformer)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, 30)
    
    def forward(self, input_ids, seg_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        pooler_output = roberta_output[1]
        out = self.classifier(pooler_output)
        return out
gc.collect()

40

# Inference

In [9]:
RoBERTa = '../input/huggingface-roberta/roberta-large'
models = []
for i in tqdm(range(5)):
    model_path = f'../input/quest5roberta/questRoBERTa_{i}.pt'
    model = QUESTModel(RoBERTa)
    model.load_state_dict(torch.load(model_path, map_location=device))
    models.append(model)
print("Loaded all models")
gc.collect()
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]Some weights of the model checkpoint at ../input/huggingface-roberta/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 20%|██        | 1/5 [00:34<02:16, 34.13s/it]Some weights of the model checkpoint at ../input/huggingface-roberta/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'l

Loaded all models


In [10]:
tokenizer = AutoTokenizer.from_pretrained(RoBERTa)
test_loader = get_test_loader(tokenizer)
all_scores = []
with torch.no_grad():
    for input_ids, seg_ids, attention_mask, in tqdm(test_loader):
        input_ids, seg_ids, attention_mask = (input_ids.to(device), seg_ids.to(device), attention_mask.to(device))
        scores = []
        for model in models:
            model.to(device)
            outputs = torch.sigmoid(model(input_ids, seg_ids, attention_mask)).cpu()
            scores.append(outputs)
            gc.collect()
            torch.cuda.empty_cache()
        all_scores.append(torch.mean(torch.stack(scores), 0))
        del input_ids, seg_ids, attention_mask
        gc.collect()
        torch.cuda.empty_cache()
all_scores = torch.cat(all_scores, 0).numpy()

100%|██████████| 119/119 [05:03<00:00,  2.55s/it]


In [11]:
all_scores[:1]

array([[0.9559347 , 0.680149  , 0.13861103, 0.64121455, 0.71505195,
        0.6812119 , 0.6785577 , 0.6648129 , 0.5767216 , 0.00407632,
        0.6072203 , 0.67201644, 0.01382279, 0.24585223, 0.01691193,
        0.01014602, 0.08064391, 0.07818709, 0.5047147 , 0.00245077,
        0.93239725, 0.90579665, 0.56464934, 0.95075834, 0.94514287,
        0.8050125 , 0.04758068, 0.04624235, 0.6657003 , 0.9085978 ]],
      dtype=float32)

In [12]:
sub[TARGET_COLUMNS] = all_scores
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.955935,0.680149,0.138611,0.641215,0.715052,0.681212,0.678558,0.664813,0.576722,0.004076,0.607220,0.672016,0.013823,0.245852,0.016912,0.010146,0.080644,0.078187,0.504715,0.002451,0.932397,0.905797,0.564649,0.950758,0.945143,0.805013,0.047581,0.046242,0.665700,0.908598
1,46,0.872893,0.445684,0.004335,0.768908,0.787627,0.917762,0.554265,0.475280,0.263074,0.002867,0.438877,0.545407,0.002198,0.001947,0.001407,0.006278,0.784905,0.200420,0.144317,0.000308,0.685778,0.938550,0.632994,0.965890,0.973074,0.853572,0.891734,0.156771,0.077292,0.866512
2,70,0.900316,0.655600,0.019553,0.793436,0.902431,0.943197,0.609926,0.474351,0.225187,0.002361,0.165988,0.873203,0.008223,0.035020,0.004502,0.008503,0.072092,0.051478,0.605010,0.001037,0.851199,0.951327,0.665617,0.969896,0.980001,0.903459,0.084306,0.065428,0.888947,0.909222
3,132,0.857022,0.454738,0.004075,0.772272,0.749260,0.914551,0.543262,0.428879,0.075250,0.007488,0.498202,0.018871,0.001638,0.001605,0.001451,0.010260,0.839469,0.204904,0.506361,0.000208,0.675748,0.940257,0.660755,0.971622,0.976392,0.862992,0.785813,0.161676,0.606887,0.889815
4,200,0.894862,0.372990,0.032051,0.795356,0.743792,0.878970,0.669179,0.626383,0.201679,0.008181,0.463369,0.682773,0.008497,0.017925,0.007398,0.025808,0.195484,0.173007,0.322233,0.001209,0.708438,0.876135,0.597725,0.944343,0.941325,0.751891,0.184839,0.141750,0.419452,0.883496


In [13]:
sub.to_csv('submission.csv', index=False)
print("Submission is ready")

Submission is ready
